# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Convert it to dataframe and clean
cities_df = pd.read_csv(output_data_file)
weather_df = cities_df.drop(columns=["Unnamed: 0"])
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,egvekinot,66.32,-179.17,18.91,94,58,9.19,RU,1603793622
1,kapaa,22.08,-159.32,69.80,88,40,6.93,US,1603793574
2,bairiki,1.33,172.98,80.60,78,20,6.93,KI,1603793622
3,esperance,-33.87,121.90,66.20,68,0,17.22,AU,1603793622
4,jamestown,42.10,-79.24,42.01,100,90,5.82,US,1603793622


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)

In [12]:
locations = weather_df[["Lat", "Lng"]]

# Get the humidity for the heatmap
humidity = weather_df["Humidity"].astype(float)
max_humidity = humidity.max()
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_humidity, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
weather_df_narrowed_df = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & 
                                        (weather_df["Cloudiness"] == 0)]

weather_df_narrowed_df = weather_df_narrowed_df.dropna(how='any')
weather_df_narrowed_df.reset_index(inplace=True)
del weather_df_narrowed_df["index"]
weather_df_narrowed_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kastamonu,41.50,33.67,71.24,30,0,2.89,TR,1603793624
1,grand gaube,-20.01,57.66,79.00,62,0,8.99,MU,1603793624
2,port elizabeth,-33.92,25.57,75.20,69,0,5.82,ZA,1603793546
3,carnarvon,-24.87,113.63,75.20,73,0,28.86,AU,1603793631
4,ardakan,32.31,54.02,75.20,13,0,6.93,IR,1603793652


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_list = []
# Set hotel_df to be the narrowed_df
hotel_df = weather_df_narrowed_df

# Time to go through each city and grab the hotel within 5000 meters and store first hotel
for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]["Lat"]
    lng = hotel_df.loc[i]["Lng"]
    
    parameters = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types": "hotel",
        "key": g_key
    }
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, parameters)
    json_conversion = response.json()
    try:
        hotel_list.append(json_conversion['results'][0]['name'])
    except:
        hotel_list.append("")

In [22]:
# Store hotel as Hotel Name in hotel_df
hotel_df["Hotel Name"] = hotel_list
hotel_df = hotel_df.dropna(how="any")
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,kastamonu,41.50,33.67,71.24,30,0,2.89,TR,1603793624,Kasaba
1,grand gaube,-20.01,57.66,79.00,62,0,8.99,MU,1603793624,Goodlands
2,port elizabeth,-33.92,25.57,75.20,69,0,5.82,ZA,1603793546,Port Elizabeth
3,carnarvon,-24.87,113.63,75.20,73,0,28.86,AU,1603793631,Carnarvon
4,ardakan,32.31,54.02,75.20,13,0,6.93,IR,1603793652,Meybod


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations)
fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))